# Finding Patients with AMI and Glucose Measurements

In [1]:
import pandas as pd

In [2]:
patient_file = r"E:\EICU\eicu-collaborative-research-database-2.0\patient.csv.gz"
patient_df = pd.read_csv(patient_file, compression="gzip")

diagnosis_file = r"E:\EICU\eicu-collaborative-research-database-2.0\diagnosis.csv.gz"
diagnosis_df = pd.read_csv(diagnosis_file, compression="gzip")

lab_file = r"E:\EICU\eicu-collaborative-research-database-2.0\lab.csv.gz"
lab_df = pd.read_csv(lab_file, compression="gzip")

In [8]:
diagnosis_df['icd9code'] = diagnosis_df['icd9code'].astype(str).str.strip()

def contains_ami(icd9_list):
    icd9_codes = [code.strip() for code in icd9_list.split(',')]
    return any(code.startswith('410') for code in icd9_codes)

ami_diag = diagnosis_df[diagnosis_df['icd9code'].apply(contains_ami)]



In [9]:
ami_diag

,diagnosisid,patientunitstayid,activeupondischarge,diagnosisoffset,diagnosisstring,icd9code,diagnosispriority
43,3588591,141266,True,96,cardiovascular|chest pain / ASHD|acute coronar...,"410.71, I21.4",Primary
367,3537048,141844,False,1995,cardiovascular|chest pain / ASHD|acute coronar...,"410.71, I21.4",Major
370,4228094,141844,False,756,cardiovascular|chest pain / ASHD|acute coronar...,"410.71, I21.4",Major
375,3821031,141844,False,54,cardiovascular|chest pain / ASHD|acute coronar...,"410.71, I21.4",Major
387,3387281,141845,True,777,cardiovascular|chest pain / ASHD|acute coronar...,"410.71, I21.4",Other
...,...,...,...,...,...,...,...
2710651,46095309,3353251,False,5521,cardiovascular|chest pain / ASHD|acute coronar...,"410.90, I21.3",Major
2710654,46095374,3353251,False,9169,cardiovascular|chest pain / ASHD|acute coronar...,"410.90, I21.3",Major
2710655,46097383,3353251,False,1201,cardiovascular|chest pain / ASHD|acute coronar...,"410.90, I21.3",Primary
2710659,46097690,3353251,False,2653,cardiovascular|chest pain / ASHD|acute coronar...,"410.90, I21.3",Primary


In [10]:
ami_df = patient_df.merge(ami_diag[['patientunitstayid']], on='patientunitstayid', how='inner')


lab_df = lab_df[(lab_df['labname'].str.contains('glucose', case=False)) & (lab_df['labresultoffset'] <= 48 * 60)]

glucose_counts = lab_df.groupby('patientunitstayid')['labresult'].count()

# 只保留至少 2 次血糖测量的患者
valid_glucose_patients = glucose_counts[glucose_counts >= 2].index
ami_df = ami_df[ami_df['patientunitstayid'].isin(valid_glucose_patients)]

In [11]:
ami_df

,patientunitstayid,patienthealthsystemstayid,gender,age,ethnicity,hospitalid,wardid,apacheadmissiondx,admissionheight,hospitaladmittime24,...,unitadmitsource,unitvisitnumber,unitstaytype,admissionweight,dischargeweight,unitdischargetime24,unitdischargeoffset,unitdischargelocation,unitdischargestatus,uniquepid
0,141266,129000,Male,73,Caucasian,71,113,"Sepsis, renal/UTI (including bladder)",188.0,21:25:00,...,Emergency Department,1,admit,120.4,112.9,22:44:00,1501,Floor,Alive,002-76877
1,141844,129452,Female,53,Caucasian,73,92,"CABG alone, coronary artery bypass grafting",162.6,23:20:00,...,Operating Room,1,admit,NaN,142.2,22:20:00,4110,Floor,Alive,002-9997
2,141844,129452,Female,53,Caucasian,73,92,"CABG alone, coronary artery bypass grafting",162.6,23:20:00,...,Operating Room,1,admit,NaN,142.2,22:20:00,4110,Floor,Alive,002-9997
3,141844,129452,Female,53,Caucasian,73,92,"CABG alone, coronary artery bypass grafting",162.6,23:20:00,...,Operating Room,1,admit,NaN,142.2,22:20:00,4110,Floor,Alive,002-9997
4,141845,129452,Female,53,Caucasian,73,92,"Efffusion, pericardial",162.6,23:20:00,...,Floor,2,readmit,NaN,135.1,19:13:00,2161,Floor,Alive,002-9997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30725,3353251,2743099,Male,73,African American,458,1104,Cardiac arrest (with or without respiratory ar...,177.8,12:51:00,...,Emergency Department,1,admit,102.0,96.2,23:16:00,16259,Telemetry,Alive,035-5166
30726,3353251,2743099,Male,73,African American,458,1104,Cardiac arrest (with or without respiratory ar...,177.8,12:51:00,...,Emergency Department,1,admit,102.0,96.2,23:16:00,16259,Telemetry,Alive,035-5166
30727,3353251,2743099,Male,73,African American,458,1104,Cardiac arrest (with or without respiratory ar...,177.8,12:51:00,...,Emergency Department,1,admit,102.0,96.2,23:16:00,16259,Telemetry,Alive,035-5166
30728,3353251,2743099,Male,73,African American,458,1104,Cardiac arrest (with or without respiratory ar...,177.8,12:51:00,...,Emergency Department,1,admit,102.0,96.2,23:16:00,16259,Telemetry,Alive,035-5166
